# Data preparation

This notebook imports and prepares the data for bakery sales prediction project. 

# Import libraries and data

Here we import 3 data tables from course [Github page](https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml.git). 

We also add new variables such as Germany's unemployment rate table between 2012 Janurary to 2019 December, downloaded from [Statistisches Bundesamt: Unemployment - German Federal Statistical Office](https://www-genesis.destatis.de/datenbank/online/statistic/13231/table/13231-0001/chart/line/search/s/ZW1wbG95bWVudA==#filter=JTdCJTIyaGlkZUVtcHR5Q29scyUyMiUzQWZhbHNlJTJDJTIyaGlkZUVtcHR5Um93cyUyMiUzQWZhbHNlJTJDJTIyY2FwdGlvbiUyMiUzQSU1QiU3QiUyMnZhcmlhYmxlSWQlMjIlM0ElMjIxMzIzMSUyMiUyQyUyMmlkJTIyJTNBJTIyZmlsdGVyLjAlMjIlMkMlMjJ2YWx1ZXNJZHMlMjIlM0ElNUIlMjIxMzIzMSUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTdCJTIydmFyaWFibGVJZCUyMiUzQSUyMkRJTlNHJTIyJTJDJTIyaWQlMjIlM0ElMjJmaWx0ZXIuMC4wJTIyJTJDJTIydmFsdWVzSWRzJTIyJTNBJTVCJTIyREclMjIlNUQlMkMlMjJjaGlsZHJlbiUyMiUzQSU1QiU1RCUyQyUyMnNob3dBc0ludGVybGluZSUyMiUzQWZhbHNlJTJDJTIyc2hvd1ZhcmlhYmxlJTIyJTNBZmFsc2UlMkMlMjJzaG93VmFyaWFibGVWYWx1ZSUyMiUzQSU1QiUyMkxBQkVMJTIyJTVEJTJDJTIyc29ydCUyMiUzQSUyMk5hbWVBc2MlMjIlMkMlMjJpc0hpZGRlbiUyMiUzQWZhbHNlJTJDJTIyYmxvY2tDb2RlJTIyJTNBJTIydjElMjIlMkMlMjJwb3NzaWJsZVBsYWNlcyUyMiUzQSU1QiU1RCU3RCU1RCUyQyUyMnNob3dBc0ludGVybGluZSUyMiUzQWZhbHNlJTJDJTIyaXNIaWRkZW4lMjIlM0FmYWxzZSUyQyUyMmJsb2NrQ29kZSUyMiUzQSUyMnMxJTIyJTJDJTIycG9zc2libGVQbGFjZXMlMjIlM0ElNUIlNUQlN0QlNUQlMkMlMjJyb3dIZWFkZXIlMjIlM0ElNUIlN0IlMjJ2YXJpYWJsZUlkJTIyJTNBJTIySkFIUiUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMCUyMiUyQyUyMnZhbHVlc0lkcyUyMiUzQSU1QiUyMjIwMTIlMjIlMkMlMjIyMDEzJTIyJTJDJTIyMjAxNCUyMiUyQyUyMjIwMTUlMjIlMkMlMjIyMDE2JTIyJTJDJTIyMjAxNyUyMiUyQyUyMjIwMTglMjIlMkMlMjIyMDE5JTIyJTVEJTJDJTIyY2hpbGRyZW4lMjIlM0ElNUIlN0IlMjJ2YXJpYWJsZUlkJTIyJTNBJTIyTU9OQVQlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAuMCUyMiUyQyUyMnZhbHVlc0lkcyUyMiUzQSU1QiUyMk1PTkFUMDElMjIlMkMlMjJNT05BVDAyJTIyJTJDJTIyTU9OQVQwMyUyMiUyQyUyMk1PTkFUMDQlMjIlMkMlMjJNT05BVDA1JTIyJTJDJTIyTU9OQVQwNiUyMiUyQyUyMk1PTkFUMDclMjIlMkMlMjJNT05BVDA4JTIyJTJDJTIyTU9OQVQwOSUyMiUyQyUyMk1PTkFUMTAlMjIlMkMlMjJNT05BVDExJTIyJTJDJTIyTU9OQVQxMiUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTVEJTJDJTIyc2hvd0FzSW50ZXJsaW5lJTIyJTNBZmFsc2UlMkMlMjJzaG93VmFyaWFibGUlMjIlM0F0cnVlJTJDJTIyc2hvd1ZhcmlhYmxlVmFsdWUlMjIlM0ElNUIlMjJMQUJFTCUyMiU1RCUyQyUyMnNvcnQlMjIlM0ElMjJDb2RlQXNjJTIyJTJDJTIyaXNIaWRkZW4lMjIlM0FmYWxzZSUyQyUyMmJsb2NrQ29kZSUyMiUzQSUyMnY0JTIyJTJDJTIycG9zc2libGVQbGFjZXMlMjIlM0ElNUIlN0IlMjJwcmV2UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjMlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0FudWxsJTJDJTIyZWxlbWVudEJlbG93JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjMlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAlMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EwJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTJDJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYzJTIyJTJDJTIyaWQlMjIlM0ElMjJyb3dUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBbnVsbCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0F0cnVlJTdEJTJDJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYzJTIyJTJDJTIyaWQlMjIlM0ElMjJyb3dUaXRsZS4wJTIyJTdEJTJDJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMxJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjAlMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EwJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQXRydWUlN0QlNUQlN0QlNUQlMkMlMjJzaG93QXNJbnRlcmxpbmUlMjIlM0FmYWxzZSUyQyUyMnNob3dWYXJpYWJsZSUyMiUzQXRydWUlMkMlMjJzaG93VmFyaWFibGVWYWx1ZSUyMiUzQSU1QiUyMkxBQkVMJTIyJTVEJTJDJTIyaXNIaWRkZW4lMjIlM0FmYWxzZSUyQyUyMmJsb2NrQ29kZSUyMiUzQSUyMnYzJTIyJTJDJTIycG9zc2libGVQbGFjZXMlMjIlM0ElNUIlN0IlMjJuZXdQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2NCUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMC4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjQlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAuMCUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQW51bGwlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EwJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTJDJTdCJTIyZWxlbWVudEFib3ZlJTIyJTNBbnVsbCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0F0cnVlJTdEJTJDJTdCJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMxJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjAlMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EwJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQXRydWUlN0QlNUQlN0QlNUQlMkMlMjJjb2x1bW5IZWFkZXIlMjIlM0ElNUIlN0IlMjJ2YXJpYWJsZUlkJTIyJTNBJTIyV0VSVDA2JTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTJDJTIydmFsdWVzSWRzJTIyJTNBJTVCJTIyV0VSVE9SRyUyMiUyQyUyMlgxM0pEVEIlMjIlMkMlMjJCVjRUQiUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTdCJTIydmFyaWFibGVJZCUyMiUzQSUyMkVSVzAwMSUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4wJTIyJTJDJTIydmFsdWVzSWRzJTIyJTNBJTVCJTIyUU1VJTIyJTVEJTJDJTIyY2hpbGRyZW4lMjIlM0ElNUIlNUQlMkMlMjJzaG93QXNJbnRlcmxpbmUlMjIlM0FmYWxzZSUyQyUyMmlzSGlkZGVuJTIyJTNBZmFsc2UlMkMlMjJibG9ja0NvZGUlMjIlM0ElMjJjMSUyMiUyQyUyMnBvc3NpYmxlUGxhY2VzJTIyJTNBJTVCJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjMiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4xJTIyJTdEJTJDJTIyZWxlbWVudEJlbG93JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzMlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMiUyMiU3RCUyQyUyMm5ld1NpYmxpbmdJbmRleCUyMiUzQTIlMkMlMjJoYXNUcmFuc3Bvc2VQYXJ0JTIyJTNBZmFsc2UlN0QlMkMlN0IlMjJwcmV2UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJuZXdQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRBYm92ZSUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMzJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjIlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjNCUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4zJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMyUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0FmYWxzZSU3RCUyQyU3QiUyMnByZXZQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMm5ld1BhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzQlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMyUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQW51bGwlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0E0JTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTVEJTdEJTJDJTdCJTIydmFyaWFibGVJZCUyMiUzQSUyMkVSVzAwMiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4xJTIyJTJDJTIydmFsdWVzSWRzJTIyJTNBJTVCJTIyUU1VJTIyJTVEJTJDJTIyY2hpbGRyZW4lMjIlM0ElNUIlNUQlMkMlMjJzaG93QXNJbnRlcmxpbmUlMjIlM0FmYWxzZSUyQyUyMmlzSGlkZGVuJTIyJTNBZmFsc2UlMkMlMjJibG9ja0NvZGUlMjIlM0ElMjJjMiUyMiUyQyUyMnBvc3NpYmxlUGxhY2VzJTIyJTNBJTVCJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMxJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjAlMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EwJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTJDJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjMyUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4yJTIyJTdEJTJDJTIyZWxlbWVudEJlbG93JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzQlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMyUyMiU3RCUyQyUyMm5ld1NpYmxpbmdJbmRleCUyMiUzQTMlMkMlMjJoYXNUcmFuc3Bvc2VQYXJ0JTIyJTNBZmFsc2UlN0QlMkMlN0IlMjJwcmV2UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJuZXdQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRBYm92ZSUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmM0JTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjMlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0FudWxsJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBNCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0FmYWxzZSU3RCU1RCU3RCUyQyU3QiUyMnZhcmlhYmxlSWQlMjIlM0ElMjJFUlcwMDMlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMiUyMiUyQyUyMnZhbHVlc0lkcyUyMiUzQSU1QiUyMlFNVSUyMiU1RCUyQyUyMmNoaWxkcmVuJTIyJTNBJTVCJTVEJTJDJTIyc2hvd0FzSW50ZXJsaW5lJTIyJTNBZmFsc2UlMkMlMjJpc0hpZGRlbiUyMiUzQWZhbHNlJTJDJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzMlMjIlMkMlMjJwb3NzaWJsZVBsYWNlcyUyMiUzQSU1QiU3QiUyMnByZXZQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMm5ld1BhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjMSUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4wJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0FmYWxzZSU3RCUyQyU3QiUyMnByZXZQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMm5ld1BhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzElMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMCUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjElMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0ExJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTJDJTdCJTIycHJldlBhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QWJvdmUlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjNCUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4zJTIyJTdEJTJDJTIyZWxlbWVudEJlbG93JTIyJTNBbnVsbCUyQyUyMm5ld1NpYmxpbmdJbmRleCUyMiUzQTQlMkMlMjJoYXNUcmFuc3Bvc2VQYXJ0JTIyJTNBZmFsc2UlN0QlNUQlN0QlMkMlN0IlMjJ2YXJpYWJsZUlkJTIyJTNBJTIyRVJXMDg5JTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjMlMjIlMkMlMjJ2YWx1ZXNJZHMlMjIlM0ElNUIlMjJRTVUlMjIlNUQlMkMlMjJjaGlsZHJlbiUyMiUzQSU1QiU1RCUyQyUyMnNob3dBc0ludGVybGluZSUyMiUzQWZhbHNlJTJDJTIyaXNIaWRkZW4lMjIlM0FmYWxzZSUyQyUyMmJsb2NrQ29kZSUyMiUzQSUyMmM0JTIyJTJDJTIycG9zc2libGVQbGFjZXMlMjIlM0ElNUIlN0IlMjJwcmV2UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJuZXdQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRBYm92ZSUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEJlbG93JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzElMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMCUyMiU3RCUyQyUyMm5ld1NpYmxpbmdJbmRleCUyMiUzQTAlMkMlMjJoYXNUcmFuc3Bvc2VQYXJ0JTIyJTNBZmFsc2UlN0QlMkMlN0IlMjJwcmV2UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAlMjIlN0QlMkMlMjJuZXdQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMmVsZW1lbnRBYm92ZSUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMxJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjAlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJjMiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMC4xJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMSUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0FmYWxzZSU3RCUyQyU3QiUyMnByZXZQYXJlbnQlMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MiUyMiUyQyUyMmlkJTIyJTNBJTIyY29sVGl0bGUuMCUyMiU3RCUyQyUyMm5ld1BhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYyJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIyYzIlMjIlMkMlMjJpZCUyMiUzQSUyMmNvbFRpdGxlLjAuMSUyMiU3RCUyQyUyMmVsZW1lbnRCZWxvdyUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMmMzJTIyJTJDJTIyaWQlMjIlM0ElMjJjb2xUaXRsZS4wLjIlMjIlN0QlMkMlMjJuZXdTaWJsaW5nSW5kZXglMjIlM0EyJTJDJTIyaGFzVHJhbnNwb3NlUGFydCUyMiUzQWZhbHNlJTdEJTVEJTdEJTVEJTJDJTIyc2hvd0FzSW50ZXJsaW5lJTIyJTNBZmFsc2UlMkMlMjJzaG93VmFyaWFibGUlMjIlM0FmYWxzZSUyQyUyMnNob3dWYXJpYWJsZVZhbHVlJTIyJTNBJTVCJTIyTEFCRUwlMjIlNUQlMkMlMjJpc0hpZGRlbiUyMiUzQWZhbHNlJTJDJTIyYmxvY2tDb2RlJTIyJTNBJTIydjIlMjIlMkMlMjJwb3NzaWJsZVBsYWNlcyUyMiUzQSU1QiU3QiUyMmVsZW1lbnRBYm92ZSUyMiUzQW51bGwlMkMlMjJlbGVtZW50QmVsb3clMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2MyUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMCUyMiU3RCUyQyUyMm5ld1NpYmxpbmdJbmRleCUyMiUzQTAlMkMlMjJoYXNUcmFuc3Bvc2VQYXJ0JTIyJTNBdHJ1ZSU3RCUyQyU3QiUyMm5ld1BhcmVudCUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnYzJTIyJTJDJTIyaWQlMjIlM0ElMjJyb3dUaXRsZS4wJTIyJTdEJTJDJTIyZWxlbWVudEFib3ZlJTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjMlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0ElN0IlMjJibG9ja0NvZGUlMjIlM0ElMjJ2NCUyMiUyQyUyMmlkJTIyJTNBJTIycm93VGl0bGUuMC4wJTIyJTdEJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0F0cnVlJTdEJTJDJTdCJTIybmV3UGFyZW50JTIyJTNBJTdCJTIyYmxvY2tDb2RlJTIyJTNBJTIydjQlMjIlMkMlMjJpZCUyMiUzQSUyMnJvd1RpdGxlLjAuMCUyMiU3RCUyQyUyMmVsZW1lbnRBYm92ZSUyMiUzQSU3QiUyMmJsb2NrQ29kZSUyMiUzQSUyMnY0JTIyJTJDJTIyaWQlMjIlM0ElMjJyb3dUaXRsZS4wLjAlMjIlN0QlMkMlMjJlbGVtZW50QmVsb3clMjIlM0FudWxsJTJDJTIybmV3U2libGluZ0luZGV4JTIyJTNBMCUyQyUyMmhhc1RyYW5zcG9zZVBhcnQlMjIlM0F0cnVlJTdEJTVEJTdEJTVEJTJDJTIyZml4Rmlyc3RDb2x1bW5zJTIyJTNBZmFsc2UlN0Q=).



In [ ]:
# import library
%pip install pandas
import pandas as pd # for data manipulation
%pip install pyodide-http
import pyodide_http # for loading external files from http 
pyodide_http.patch_all() 
%pip install matplotlib
import matplotlib.pyplot as plt # for plottings 
%pip install holidays # for holidays features
import holidays

# import data 
weather = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv')
kiwo = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv')
sale = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv')
unemployment = pd.read_csv("0_DataPreparation/input_data/13231-0001_en_flat.csv", sep= ';')

# Data cleaning 

We format the unemployment rate table and extract year, month, unemployment rate value into new table. 

We expand the unemployment data so each day in a month gets the same unemployment value. This way, we can merge on the exact date with other tables. 

In [47]:
# Select relevant columns for unemployment rate
unemployment_clean = unemployment[
	["time", "1_variable_attribute_code", "3_variable_attribute_label", "value_variable_label", "value", "value_unit"]
]
# Filter for rows where value_variable_label is 'Unemployment rate' and 3_variable_attribute_label is 'BV4.1 trend'
unemployment_clean = unemployment_clean[
	(unemployment_clean["value_variable_label"] == "Unemployment rate") &
	(unemployment_clean["3_variable_attribute_label"] == "BV4.1 trend")
]

# Sort the DataFrame by 'time' and '1_variable_attribute_code'
unemployment_clean = unemployment_clean.sort_values(by=["time", "1_variable_attribute_code"])

# Create new columns for year and month
# Extract year from "time"
unemployment_clean["year"] = unemployment_clean["time"].astype(int)

# Extract month number from "1_variable_attribute_code" (e.g., "MONAT01" -> 1)
unemployment_clean["month"] = unemployment_clean["1_variable_attribute_code"].str.extract(r"MONAT(\d{2})").astype(int)

# Create a date column as the first day of each month
unemployment_clean["date"] = pd.to_datetime(
    unemployment_clean["year"].astype(str) + "-" + unemployment_clean["month"].astype(str).str.zfill(2) + "-01"
)

# Now you have a 'date' column like 2012-01-01, 2012-02-01, etc.
unemployment_clean[["time", "1_variable_attribute_code", "year", "month", "date", "value"]].head()

# Expand unemployment_clean so each day in the month gets the same value
expanded_rows = []
for _, row in unemployment_clean.iterrows():
    # Get all days in the month
    days = pd.date_range(start=row['date'], periods=1, freq='MS')  # first day of month
    last_day = (row['date'] + pd.offsets.MonthEnd(0)).day
    days = pd.date_range(start=row['date'], periods=last_day, freq='D')
    for day in days:
        expanded_rows.append({'Datum': day, 'Unemployment_Rate': row['value']})

unemployment_daily = pd.DataFrame(expanded_rows)

unemployment_clean.head()
unemployment_daily.head()

,Datum,Unemployment_Rate
0,2012-01-01,5.2
1,2012-01-02,5.2
2,2012-01-03,5.2
3,2012-01-04,5.2
4,2012-01-05,5.2


# Data Merging

We merge all tables into one dataframe by "Datum". 

In [53]:
# merge data 
#sale_weather_kiwo = sale.merge(weather, on='Datum', how='outer', suffixes = ("_sale", "_weather")) \
#.merge(kiwo, on='Datum', how='outer', suffixes = ("", "_kiwo"))

sale_weather = pd.merge_ordered(sale, weather, on='Datum', suffixes = ("_sale", "_weather"), how='outer')
sale_weather_kiwo = pd.merge_ordered(sale_weather, kiwo, on='Datum', suffixes = ("", "kiwo"), how='outer')        

# Ensure 'Datum' is datetime for merging
sale_weather_kiwo['Datum'] = pd.to_datetime(sale_weather_kiwo['Datum'], errors='coerce')

# Merge with your main dataframe on 'Datum'
sale_weather_kiwo = sale_weather_kiwo.merge(unemployment_daily, on='Datum', how='left')

sale_weather_kiwo.head()

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Unemployment_Rate
0,NaN,2012-01-01,NaN,NaN,8.0,9.8250,14.0,58.0,NaN,5.2
1,NaN,2012-01-02,NaN,NaN,7.0,7.4375,12.0,NaN,NaN,5.2
2,NaN,2012-01-03,NaN,NaN,8.0,5.5375,18.0,63.0,NaN,5.2
3,NaN,2012-01-04,NaN,NaN,4.0,5.6875,19.0,80.0,NaN,5.2
4,NaN,2012-01-05,NaN,NaN,6.0,5.3000,23.0,80.0,NaN,5.2


# Data cleaning and feature engineering

Here we removed rows with missing data from the merged dataset `sale_weather_kiwo`, then we categorized different features such as temperature, cloud coverage, holidays, and seasons. 

In [58]:
# === DATE-BASED FEATURES - categorizing "datum" values ===
sale_weather_kiwo['Datum'] = pd.to_datetime(sale_weather_kiwo['Datum'], errors='coerce')
sale_weather_kiwo = sale_weather_kiwo.dropna(subset=['Datum'])  # Remove rows with missing date
sale_weather_kiwo = sale_weather_kiwo.sort_values(by='Datum') 

sale_weather_kiwo['DayOfWeek'] = sale_weather_kiwo['Datum'].dt.dayofweek
sale_weather_kiwo['Weekend'] = sale_weather_kiwo['DayOfWeek'].isin([5, 6]).astype(int)
sale_weather_kiwo['Month'] = sale_weather_kiwo['Datum'].dt.month
sale_weather_kiwo['Year'] = sale_weather_kiwo['Datum'].dt.year
sale_weather_kiwo['Week'] = sale_weather_kiwo['Datum'].dt.isocalendar().week
sale_weather_kiwo['Day'] = sale_weather_kiwo['Datum'].dt.day

sale_weather_kiwo['KielerWoche'] = sale_weather_kiwo['KielerWoche'].fillna(0).astype(int)

# === WEATHER-BASED FEATURES ===

# Categorize temperature
temp_bins = [-float("inf"), 0, 10, 20, 30, float("inf")]
temp_labels = ["very cold", "cold", "mild", "warm", "hot"]
sale_weather_kiwo['TemperatureCategory'] = pd.cut(sale_weather_kiwo['Temperatur'], bins=temp_bins, labels=temp_labels)

# Categorize cloud cover
cloud_bins = [-1, 2, 5, 8, float("inf")]
cloud_labels = ["clear", "partly cloudy", "cloudy", "overcast"]
sale_weather_kiwo['CloudCategory'] = pd.cut(sale_weather_kiwo['Bewoelkung'], bins=cloud_bins, labels=cloud_labels)

# Categorize wind speed
wind_bins = [-1, 10, 20, 30, float("inf")]
wind_labels = ["light", "moderate", "strong", "very strong"]
sale_weather_kiwo['WindCategory'] = pd.cut(sale_weather_kiwo['Windgeschwindigkeit'], bins=wind_bins, labels=wind_labels)

# === HOLIDAYS FEATURES === 
# Categorize holidays using 'holiday' library
german_holidays = holidays.Germany(state='SH')  # SH = Schleswig-Holstein
sale_weather_kiwo['IsHoliday_lib'] = sale_weather_kiwo['Datum'].apply(
    lambda x: 1 if x.date() in german_holidays else 0
)
sale_weather_kiwo['HolidayName'] = sale_weather_kiwo['Datum'].apply(
    lambda x: german_holidays.get(x.date(), 'No Holiday')
)

# === SEASONS FEATURES: month-based and temperature as backup ===
def get_season_month_temp_backup(date, temp):
    """
    Define seasons based on month first, with temperature as backup/alternative
    
    Primary criteria (month-based):
    - Winter: months 12, 1, 2
    - Spring: months 3, 4, 5  
    - Summer: months 6, 7, 8
    - Autumn: months 9, 10, 11
    
    Backup criteria (temperature-based):
    - Winter: temp ≤ 5°C
    - Spring: temp 5-12°C
    - Summer: temp 12-18°C
    - Autumn: temp > 18°C
    """
    month = date.month
    
    # Primary: Month-based classification
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    
    # Backup: If somehow month is not in expected range, use temperature
    if not pd.isna(temp):
        if temp <= 5:
            return 'Winter'
        elif 5 < temp <= 12:
            return 'Spring'
        elif 12 < temp <= 18:
            return 'Summer'
        else:  # temp > 18
            return 'Autumn'
    
    # Final fallback
    return 'Unknown'

# Apply month-based season classification with temperature backup
sale_weather_kiwo['Season'] = sale_weather_kiwo.apply(
    lambda row: get_season_month_temp_backup(row['Datum'], row['Temperatur']), axis=1
)

# view the merged and prepared data
sale_weather_kiwo.head()


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Unemployment_Rate,...,Month,Year,Week,Day,TemperatureCategory,CloudCategory,WindCategory,IsHoliday_lib,HolidayName,Season
0,NaN,2012-01-01,NaN,NaN,8.0,9.8250,14.0,58.0,0,5.2,...,1,2012,52,1,cold,cloudy,moderate,1,Neujahr,Winter
1,NaN,2012-01-02,NaN,NaN,7.0,7.4375,12.0,NaN,0,5.2,...,1,2012,1,2,cold,cloudy,moderate,0,No Holiday,Winter
2,NaN,2012-01-03,NaN,NaN,8.0,5.5375,18.0,63.0,0,5.2,...,1,2012,1,3,cold,cloudy,moderate,0,No Holiday,Winter
3,NaN,2012-01-04,NaN,NaN,4.0,5.6875,19.0,80.0,0,5.2,...,1,2012,1,4,cold,partly cloudy,moderate,0,No Holiday,Winter
4,NaN,2012-01-05,NaN,NaN,6.0,5.3000,23.0,80.0,0,5.2,...,1,2012,1,5,cold,cloudy,strong,0,No Holiday,Winter


# Split dataset 

Here we split the merged data into training, validation and test set based on dates. 


In [ ]:
# === TIME SERIES SPLIT ===
# Split your dataset into a training set from 01.07.2013 to 31.07.2017, a validation set from 01.08.2017 to 31.07.2018, and a test set from 01.08.2018 to 31.07.2019.
train_start = '2013-07-01'
train_end = '2017-07-31'
val_start = '2017-08-01'
val_end = '2018-07-31'
test_start = '2018-08-01'
test_end = '2019-07-31'

train_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= train_start) & (sale_weather_kiwo['Datum'] <= train_end)]
val_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= val_start) & (sale_weather_kiwo['Datum'] <= val_end)]
test_data = sale_weather_kiwo[(sale_weather_kiwo['Datum'] >= test_start) & (sale_weather_kiwo['Datum'] <= test_end)]

print("Rows:", len(train_data))
print("Rows:", len(val_data))
print("Rows:", len(test_data))


# Export merged and cleaned data

In [57]:
# Save the prepared data to a CSV file
sale_weather_kiwo.to_csv("0_DataPreparation/processed_data/prepared_sales_weather_event_data.csv", index=False)
sale_weather_kiwo.to_csv("prepared_sales_weather_event_data.csv", index=False)